In [1]:
!pip install transformers peft bitsandbytes accelerate datasets trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 20.8 MB/s eta 0:00:00


Lab 1: PERF (Full Fine-tuning)

In [5]:
from datasets import Dataset

data_math = [
    {"text": "Q: 12 * 7\nA: 84"},
    {"text": "Q: 5 + 9\nA: 14"},
]

dataset_math = Dataset.from_list(data_math)
# dataset_math = dataset_math.shuffle(seed=42).select(range(200))
# dataset_math = Dataset.from_list(data_math)

N = 200
dataset_math = Dataset.from_list(data_math)
dataset_math = dataset_math.shuffle(seed=42).select(
    range(min(N, len(dataset_math)))
)

In [8]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    device_map="auto",
    torch_dtype="float16"
)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

🔬 Lab 2: LoRA

In [9]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


🔬 Lab 3: QLoRA (🔥 สำคัญมาก)

In [12]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch, gc

gc.collect()
torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto",
    max_memory={0: "13GB"},   # 👈 สำคัญมากสำหรับ T4
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 110.12 MiB is free. Process 3118 has 14.63 GiB memory in use. Of the allocated memory 14.43 GiB is allocated by PyTorch, and 87.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Lab 4: DLoRA (Dynamic / Multi-Adapter LoRA)

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1566: UserWarning: Current model requires 32 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)


In [15]:
from peft import get_peft_model

model = get_peft_model(model, lora_config)

# เพิ่ม adapters
model.add_adapter("task_math", lora_config)
model.add_adapter("task_chat", lora_config)
model.add_adapter("task_code", lora_config)

model.print_trainable_parameters()


trainable params: 2,621,440 || all params: 2,790,169,600 || trainable%: 0.0940


In [16]:
from transformers import AutoTokenizer
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_math = dataset_math.map(tokenize, batched=True)
tokenized_math = tokenized_math.shuffle(seed=42)



Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [19]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# training_args = TrainingArguments(
#     output_dir="./dloras/math",
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=4,
#     num_train_epochs=1,
#     fp16=True,
#     logging_steps=1,
#     save_strategy="no",
#     report_to="none"
# )

training_args = TrainingArguments(
    output_dir="./dloras/math",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,

    bf16=True,      # ✅ ใช้ bf16
    fp16=False,     # ❌ ปิด fp16

    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_math,
    data_collator=data_collator
)


The model is already on multiple devices. Skipping the move to device specified in `args`.


In [21]:
from peft import PeftModel

print(type(model))


<class 'peft.peft_model.PeftModelForCausalLM'>


In [22]:
model.set_adapter("task_math")
trainer.train()
model.save_pretrained("dloras/math", "task_math")



Step,Training Loss
1,2.307000
2,2.153500
3,1.858300


In [23]:
model.set_adapter("task_chat")
trainer.train()
model.save_pretrained("dloras/chat", "task_chat")


Step,Training Loss
1,2.618000
2,2.837100
3,2.666500


In [25]:
from google.colab import files
files.download("dloras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>